# Basic syntactic sugar

We group here some small modern features, added to ease your developer life.

## At last, `>>` allowed for templates  of templates

When nesting template arguments, no more need to insert a space between subsequent "<" or ">".

## At last, default member variable initialization

In a class or struct definition, one can now give a default value to any member variable.

In [2]:
#include <iostream>

In [3]:
struct MyInt {
    MyInt() {}
    MyInt( int x ) : m_x(x) {}
    int m_x = 42 ; // default value
} ;

In [5]:
MyInt i ;
MyInt j(1) ;
std::cout << i.m_x << std::endl ;
std::cout << j.m_x << std::endl ;

42
1


## Preview of the keyword `auto`

Tired of typing those painful iterator types, although what you want is so obvious ? When defining a variable with an initial value, you can now let the C++11 compiler deduce the type : just say `auto` instead of the type name. Would you guess how to do it below ?

In [6]:
#include <list>
#include <map>

In [7]:
std::map<std::vector<int>,std::list<float>> m ;
std::map<std::vector<int>,std::list<float>>::iterator itr = m.begin() ;

## Preview of the "range-based for"

A new flavor of `for` enables to process simply all the elements of a collection, provided this collection has methods `begin()` and `end()`. Instead of:

In [8]:
const int MAX = 5 ;
double values[MAX] = { 1.1, 2.2, 3.3, 4.4, 5.5 } ;
for ( int indice = 0 ; indice <MAX ; ++indice  )
 { std::cout << values[indice] << " " ; }
std::cout << std::endl ;

1.1 2.2 3.3 4.4 5.5 


One can now write:

In [9]:
const int MAX = 5 ;
double values[MAX] = { 1.1, 2.2, 3.3, 4.4, 5.5 } ;
for ( double value : values  )
  { std::cout << value << " " ; }
std::cout << std::endl ;

1.1 2.2 3.3 4.4 5.5 


If you combine with `auto`, instead of:

In [10]:
std::vector<int> values(5,0) ;
std::vector<int>::iterator itr ;
for ( itr = values.begin() ; itr != values.end() ; ++itr )
 { std::cout << *itr << " " ; }
std::cout << std::endl ;

0 0 0 0 0 


One can now write:

In [11]:
std::vector<int> values(5,0) ;
for ( auto value : values )
 { std::cout << value << " " ; }
std::cout << std::endl ;

0 0 0 0 0 


## Scoped enumerations

In ancient C++, the enumerations let you define constants which are **considered as integers**. This is both handy and sometimes tricky. Try the code below.

In [12]:
enum color { red, green, blue } ;
enum shape { circle, square, triangle } ;

In [13]:
bool is_red_triangle( int color, int shape ) {
    return ((color==red)&&(shape==triangle)) ;
}

In [14]:
#include <iostream>

In [15]:
shape s = triangle ;
color c = red ;
if (is_red_triangle(s,c))
  std::cout << "This is a red triangle !" << std::endl ;
else
  std::cout << "This is NOT a red triangle !" << std::endl ;

This is NOT a red triangle !


C++11 let you define "scoped enumerations", with the keywords `enum class`, which are not any more equated to integers, but **independant types**. Their use is a bit more verbose, but a lot less error-prone.

In [ ]:
enum class Color { yellow=5, red, blue, green } ;
Color c1, c2 ;
//.....
c1 = Color::red ;                        // OK
c2 = Color::yellow+3 ;                   // Forbidden
if (c1==Color::red) std::cout<<"red" ;   // OK
if (c1==6) std::cout<<"red" ;            // Forbidden
if (c2>Color::blue) std::cout<<"green" ; // Forbidden

## User-defined literals

C++ uses suffixes for its literals for a long time. For example, the ending `L` in `42L` means that the type is `long int`. C++11 introduces the ability to define new suffixes of format `_something` (*user-defined litterals*). In simple terms, when dealing with numbers, one has to overload `operator""_something` for the most detailed type, i.e `long double` and/or `unsigned long long`.

For example, this is the way to define a suffix `_K` able to multiply by 1000 the integer and floating numbers (not supported by the notebook kernel):

In [16]:
%%file tmp.syntactic-sugar.cpp

#include <iostream>

long double operator ""_K ( long double value )
 { return 1000.*value ; }

unsigned long long operator ""_K ( unsigned long long value )
 { return 1000*value ; }

int main()
 {
  int var = -42_K ;
  std::cout<<var<<std::endl ;
  std::cout<<3.14_K<<std::endl ;
 }

Writing tmp.syntactic-sugar.cpp


In [22]:
!rm -f tmp.syntactic-sugar.exe && g++ -std=c++17 tmp.syntactic-sugar.cpp -o tmp.syntactic-sugar.exe

In [23]:
!./tmp.syntactic-sugar.exe

-42000
3140


In the function above, we use the same types for input and output, but we could use any type for the output, and we usually use "stronger" types.

## Pointer to nothing

C++03 offers no satisfying way to express "nothing" for a pointer. One may use the macro NULL coming from C, or define  a constant integer `NULLPTR` equal to 0, or directly use 0... but the compiler will always consider it as an integer, and badly choose among overloaded functions.

Below, we would prefer the compiler to select `fnul( char * )`, but the compiler will not.

In [1]:
#include <iostream>

In [2]:
void fnul(char *) { std::cout<<"char *"<<std::endl ; }

In [3]:
void fnul(int) { std::cout<<"int"<<std::endl ; }

In [4]:
void fnul(long) { std::cout<<"long"<<std::endl ; }

In [5]:
const int NULLPTR = 0 ;
fnul(0) ;
fnul(NULLPTR) ;
fnul(NULL) ; // implementation dependant

int
int
long


You may declare `NULLPTR` of type `void *`, but the conversion from `void *` into `char *` is not implicit, and an explicit cast will be needed each time `NULLPTR` is used :

In [6]:
void * const NULLPTR = (void *)0 ;
fnul((char *)NULLPTR) ; // appel de f(char *)

char *


At last, C++11 introduce the keyword `nullptr`, which play the role of a pointer literal. As awaited, it cannot be implicitly converted into an integer, and it can be implictly converted into any kind of pointer, so the following code will select the right function:

In [7]:
fnul(nullptr) ;

char *


`nullptr` cannot be implicitly converted into an integer, but it can be into a boolean.

## Exercise

Simplify or improve the code below, thanks to new features shown above.

In [ ]:
%%file tmp.syntactic-sugar.cpp

#include <cassert>
#include <iostream>
#include <list>

template< typename Real >
class Point
 {
  public :
    Point( Real x, Real y )
     : m_x(x), m_y(y) {}
    friend std::ostream & operator<<( std::ostream & os, Point const & p )
     { return (os<<"("<<p.m_x<<","<<p.m_y<<")") ; }
  private :
    Real m_x, m_y ;
 } ;

enum BrokenLineKind { Empty, Opened, Closed } ;

template< typename Real >
class BrokenLine
 {
  public :
    BrokenLine()
     : m_kind(Empty) {}
    void add( Point<Real> p )
     { m_points.push_back(p) ; m_kind = Opened ; }
    void close()
     { assert(m_points.size()>0) ; m_kind = Closed ; }
    friend std::ostream & operator<<( std::ostream & os, BrokenLine const & bl )
     {
      char sep = '(' ;
      typename std::list< Point<Real> >::const_iterator pitr ;
      for ( pitr = bl.m_points.begin() ; pitr != bl.m_points.end() ; ++pitr )
       {  os<<sep<<(*pitr) ; sep = ',' ; }
      if (bl.m_kind==Closed)
       { os<<sep<<bl.m_points.front() ; }
      return (os<<')') ;
     }
  private :
    std::list< Point<Real> > m_points ;
    int m_kind ;
 } ;
 
const double KM = 1000.0 ;
const double M = 1.0 ;
const double MM = 0.001 ;

int main()
 {
  BrokenLine<double> bl ;
  bl.add(Point<double>(1000.0*MM,1.0*M)) ;
  bl.add(Point<double>(0.002*KM,3000.0*MM)) ;
  bl.add(Point<double>(4.0*M,0.002*KM)) ;
  bl.close() ;
  std::cout<<bl<<std::endl ;
 }

In [ ]:
!rm -f tmp.syntactic-sugar.exe && g++ -std=c++17 tmp.syntactic-sugar.cpp -o tmp.syntactic-sugar.exe

In [ ]:
!./tmp.syntactic-sugar.exe

---
© *CNRS 2020*  
*Assembled and written by David Chamont, this work is made available according to the terms of the*  
[*Creative Commons License - Attribution - NonCommercial - ShareAlike 4.0 International*](http://creativecommons.org/licenses/by-nc-sa/4.0/)

---
## Annexes

#### New type `long long int`

In a way compatible with C99, the new integer type `long long int` has a size of at least 64 bits.

#### C++17: Selection Initialization

The `if` and `switch` statements can now receive an optional initialization part, similar to the one used within `for` statements:

```c++
if ( int res = compute() ; res != 0 ) {
    // use res
}
```